In [ ]:
# (Optional) Recreate environment — keep commented unless needed
# !pip install numpy xarray rasterio shapely pyproj matplotlib panel holoviews hvplot tqdm jupyter_bokeh pystac-client
# HyTools variants (optional; choose one if needed):
# !pip install "git+https://github.com/EnSpec/hytools.git"


In [ ]:
# === Imports (kept together per spec) ===
import os, json, math
from pathlib import Path
from datetime import datetime, date, timezone, timedelta

import numpy as np
import rasterio
import matplotlib.pyplot as plt
from shapely.geometry import Point, box
from tqdm import tqdm

import panel as pn
import holoviews as hv
import hvplot.xarray  # registers accessor

from pystac_client import Client

# S2 helpers (expects your local package 's2_data' to be available)
from s2_data.s2_utils import (
    point_buffer_bbox, get_item_dt_utc, download_asset, plot_s2_truecolor
)
from s2_data.cloud_utils import (
    find_asset_key, reproject_geom, scl_metrics, count_cloud_pixels
)

# EMIT helpers (expects your local package 'EMIT_data' to be available)
from EMIT_data.EMIT_utils import (
    login, search, download_reflectance, open_reflectance, attach_wavelengths
)
from EMIT_data.emit_tools import quality_mask, band_mask
from EMIT_data.emit_proj import nc_to_envi
from EMIT_data.viz import show_emit_rgb_from_envi

pn.extension("tabulator")
hv.extension("bokeh")


## Configure Location & Parameters (Step 2)

In [ ]:
# === Location & knobs ===
# Change these two to your target location (lon/lat):
LAT = 50.3794
LON = 45.1656

# ROI search settings
SEARCH_BUFFER_M = 5000  # meters radius around point

# STAC / S2
S2_API = "https://earth-search.aws.element84.com/v1"
S2_COLLECTION = "sentinel-2-l2a"

# Date window to enumerate EMIT scenes (S2 matched per SAME date)
from datetime import date
DATE_START = date(2022, 1, 1)
DATE_END = date.today()

# Cloud thresholds
MAX_S2_CLOUD_FRAC = 50.0  # 0.0 => only fully cloud-free S2 candidates
MAX_EMIT_CLOUD_PCT = None # e.g., 5.0 to require <= 5% EMIT clouds; None => keep min per date only

# Output folders
ROOT = Path("pairs_output")
EMIT_DIR = ROOT / "emit"
S2_DIR   = ROOT / "s2"
ENVI_DIR = ROOT / "envi"
FIG_DIR  = ROOT / "figures"
for p in (EMIT_DIR, S2_DIR, ENVI_DIR, FIG_DIR):
    p.mkdir(parents=True, exist_ok=True)


## Helper Functions

In [ ]:
from datetime import timezone

def _emit_item_date(item) -> date:
    """Extract UTC date from an EMIT search item."""
    try:
        iso = item["umm"]["ProviderDates"][0]["Date"]
    except Exception:
        iso = item.get("datetime") or item.get("start_time")
    dt_utc = datetime.fromisoformat(str(iso).replace("Z", "+00:00")).astimezone(timezone.utc)
    return dt_utc.date()

def _emit_cloud_pct(item) -> float:
    try:
        return float(item["umm"].get("CloudCover"))
    except Exception:
        return float("inf")

def find_emit_candidates(lon: float, lat: float):
    """Search EMIT around the location and within DATE_START..DATE_END."""
    login(persist=True)
    roi_bbox = point_buffer_bbox(lon, lat, SEARCH_BUFFER_M)
    items = search(
        point=(lon, lat),
        bbox=roi_bbox,
        buffer_m=SEARCH_BUFFER_M,
        start=DATE_START.isoformat(),
        end=DATE_END.isoformat(),
        cloud_cover=(0.0, 1.0),
    )
    return list(items)

def choose_best_emit_per_date(items):
    """Group by date and keep least-cloudy EMIT per date; apply optional max cloud threshold."""
    by_date = {}
    for it in items:
        d = _emit_item_date(it)
        cur = by_date.get(d)
        if cur is None or _emit_cloud_pct(it) < _emit_cloud_pct(cur):
            by_date[d] = it
    if MAX_EMIT_CLOUD_PCT is not None:
        by_date = {d: it for d, it in by_date.items() if _emit_cloud_pct(it) <= MAX_EMIT_CLOUD_PCT}
    return {d.isoformat(): it for d, it in by_date.items()}

def find_best_s2_for_date(date_iso: str, lon: float, lat: float):
    """For the SAME date, search S2 in ROI; return least-cloudy item + cloud fraction using SCL."""
    bbox = point_buffer_bbox(lon, lat, SEARCH_BUFFER_M)
    roi_geom = box(*bbox.bounds)
    time_range = f"{date_iso}T00:00:00Z/{date_iso}T23:59:59Z"
    client = Client.open(S2_API)
    search = client.search(collections=[S2_COLLECTION], datetime=time_range, bbox=bbox.bounds)
    items = list(search.get_items())
    if not items:
        return None, None

    best_item = None
    best_frac = None
    for item in tqdm(items, desc=f"S2 cloud check {date_iso}"):
        key = find_asset_key(item.assets, ["scl", "scl-jp2"])
        asset = item.assets[key]
        url = asset.href
        ext = Path(url).suffix or (".tif" if key == "scl" else ".jp2")
        scl_path = S2_DIR / f"{item.id}_SCL{ext}"
        if not scl_path.exists():
            download_asset(url, scl_path)
        try:
            clouds, total = count_cloud_pixels(str(scl_path), roi_geom)
        except rasterio.errors.RasterioIOError:
            continue
        frac = (clouds / total) if total else 1.0
        if best_frac is None or frac < best_frac:
            best_frac = frac
            best_item = item
    return best_item, best_frac

def download_s2_truecolor(item) -> Path:
    """Download S2 visual (true color) if available; else save B04/B03/B02 and return JSON list."""
    assets = item.assets
    if "visual" in assets:
        href = assets["visual"].href
        out = S2_DIR / f"{item.id}_visual.tif"
        if not out.exists():
            download_asset(href, out)
        return out
    band_paths = []
    for b in ("B04", "B03", "B02"):
        if b in assets:
            href = assets[b].href
            out = S2_DIR / f"{item.id}_{b}.tif"
            if not out.exists():
                download_asset(href, out)
            band_paths.append(str(out))
    out_json = S2_DIR / f"{item.id}_RGB_bands.json"
    out_json.write_text(json.dumps(band_paths, indent=2))
    return out_json


def convert_emit_nc_to_envi(emit_nc_path: Path, s2_visual_path: Path, out_dir: Path, emit_obs_nc) -> Path:
    out_dir.mkdir(parents=True, exist_ok=True)
    tmp_dir = out_dir / "tmp"
    tmp_dir.mkdir(parents=True, exist_ok=True)
    print(emit_nc_path)
    nc_to_envi(
        img_file=emit_nc_path[0],
        out_dir=str(out_dir),
        temp_dir=str(tmp_dir),
        obs_file=emit_obs_nc,
        export_loc=True,
        crid="000",
        s2_tif_path=str(s2_visual_path),
        match_res=False,
        write_xml=False,
    )
    for p in sorted(out_dir.glob("**/*RFL*_*.bin")):
        return p
    any_bin = next(out_dir.glob("**/*.bin"), None)
    if any_bin:
        return any_bin
    raise FileNotFoundError(f"ENVI .bin not found in {out_dir}")

def visualize_pair(date_iso: str, s2_path: Path, envi_bin_path: Path) -> Path:
    fig = plt.figure(figsize=(12,5))
    ax1 = fig.add_subplot(1,2,1)
    ax2 = fig.add_subplot(1,2,2)
    try:
        plot_s2_truecolor(str(s2_path), ax=None)
    except Exception as e:
        ax1.text(0.5, 0.5, f"S2 plot failed: {e}", ha="center")
        ax1.set_axis_off()
    try:
        show_emit_rgb_from_envi(str(envi_bin_path.parent), gamma=1.0, pattern=envi_bin_path.name, white_balance=False)
        # ax2.text(0.5, 0.5, "Displayed via notebook UI", ha="center")
        # ax2.set_axis_off()
        # ax2.set_title("EMIT RGB (ENVI)")
    except Exception as e:
        ax2.text(0.5, 0.5, f"EMIT plot failed: {e}", ha="center")
        ax2.set_axis_off()
    out_png = FIG_DIR / f"pair_{date_iso}.png"
    fig.tight_layout(); fig.savefig(out_png, dpi=150); plt.close(fig)
    return out_png


In [ ]:
from pprint import pprint
import pandas as pd

roi_bbox = point_buffer_bbox(LON, LAT, SEARCH_BUFFER_M)
print("ROI BBOX:", roi_bbox)

In [ ]:
# 3) Enumerate EMIT by date
picks = search(point=(LON, LAT), bbox = roi_bbox.bounds, buffer_m=SEARCH_BUFFER_M, start=None, end=None, cloud_cover = (0.0,100.0))
emit_by_date = choose_best_emit_per_date(picks)
print(f"Found {len(emit_by_date)} EMIT dates")

In [ ]:
pairs = []
for d_iso, emit_item in emit_by_date.items():
    s2_item, s2_cloud = find_best_s2_for_date(d_iso, LON, LAT)
    if s2_item is None:
        continue
    if (s2_cloud is None) or (s2_cloud > MAX_S2_CLOUD_FRAC):
        continue
    pairs.append({
        "date": d_iso,
        "emit": emit_item,
        "emit_cloud_pct": _emit_cloud_pct(emit_item),
        "s2": s2_item,
        "s2_cloud_frac": s2_cloud,
    })

print(f"Kept {len(pairs)} EMIT/S2 same-day pairs after cloud filtering.")

In [ ]:
manifest = []
for pair in tqdm(pairs, desc="Downloading & converting"):
    d_iso = pair["date"]
    # emit_paths = download_reflectance(pair["emit"], str(EMIT_DIR), assets=['_RFL_']) 
    # mask_paths = download_reflectance(pair["emit"], str(EMIT_DIR), assets=['_MASK_'])
    # print(emit_paths)
    # s2_tc   = download_s2_truecolor(pair["s2"])

    # if isinstance(s2_tc, Path) and s2_tc.suffix.lower() == ".tif":
    #     out_dir = ENVI_DIR / f"emit_{d_iso}"
    #     envi_bin = convert_emit_nc_to_envi(emit_paths, s2_tc, out_dir, emit_obs_nc = mask_paths[0])
    # else:
    #     out_dir = ENVI_DIR / f"emit_{d_iso}"
    #     out_dir.mkdir(parents=True, exist_ok=True)
    #     envi_bin = None
    

    fig_path = None
    if isinstance(s2_tc, Path) and envi_bin is not None:
        try:
            fig_path = visualize_pair(d_iso, s2_tc, envi_bin)
        except Exception:
            pass

    entry = {
        "date": d_iso,
        "emit_id": pair["emit"].get("id", None) or pair["emit"].get("meta", {}).get("id"),
        "emit_nc": str(emit_paths),
        "emit_cloud_pct": pair.get("emit_cloud_pct"),
        "s2_id": pair["s2"].id,
        "s2_cloud_frac": pair.get("s2_cloud_frac"),
        "s2_visual": str(s2_tc),
        "envi_dir": str(out_dir),
        "envi_bin": str(envi_bin) if envi_bin else None,
        "figure": str(fig_path) if fig_path else None,
    }
    manifest.append(entry)

In [ ]:
manifest = []
for pair in tqdm(pairs, desc="Downloading & converting"):
    d_iso = pair["date"] 

    fig_path = None
    if isinstance(s2_tc, Path) and envi_bin is not None:
        try:
            fig_path = visualize_pair(d_iso, s2_tc, envi_bin)
        except Exception:
            pass

    entry = {
        "date": d_iso,
        "emit_id": pair["emit"].get("id", None) or pair["emit"].get("meta", {}).get("id"),
        "emit_nc": str(emit_paths),
        "emit_cloud_pct": pair.get("emit_cloud_pct"),
        "s2_id": pair["s2"].id,
        "s2_cloud_frac": pair.get("s2_cloud_frac"),
        "s2_visual": str(s2_tc),
        "envi_dir": str(out_dir),
        "envi_bin": str(envi_bin) if envi_bin else None,
        "figure": str(fig_path) if fig_path else None,
    }
    manifest.append(entry)

In [ ]:

manifest_path = ROOT / "pairs_manifest.json"
manifest_path.write_text(json.dumps(manifest, indent=2))
print(f"Saved manifest → {manifest_path}")

# Display a tidy summary
if manifest:
    df = pd.DataFrame(manifest)
    display(df[[
        "date", "emit_id", "emit_cloud_pct", "s2_id", "s2_cloud_frac", "envi_bin", "figure"
    ]])
else:
    print("No pairs were produced under current thresholds. Consider relaxing MAX_S2_CLOUD_FRAC or expanding date range.")


In [ ]:
from pathlib import Path
import netCDF4 as nc, os, textwrap

p = Path("pairs_output/emit/EMIT_L2A_RFL_001_20240410T101105_2410107_012.nc")
print("exists?", p.exists())
print("size bytes:", p.stat().st_size if p.exists() else None)

# Check HDF5 magic number
def is_hdf5(path: Path) -> bool:
    try:
        with open(path, "rb") as f:
            return f.read(8) == b"\x89HDF\r\n\x1a\n"
    except Exception:
        return False

print("HDF5 signature?", is_hdf5(p))

# Try to open and show a better hint
try:
    with nc.Dataset(str(p)) as ds:
        print("Variables sample:", list(ds.variables)[:10])
except Exception as e:
    print("netCDF open error:", e)
    try:
        head = open(p, "rb").read(200).decode("utf-8", errors="ignore")
        print("File head:", textwrap.shorten(head.replace("\n"," "), width=160))
    except Exception as ee:
        print("Could not read head:", ee)

# Also check for hidden whitespace/newlines in the string you pass:
img_file = str(p)
print("repr(img_file) ->", repr(img_file))
